# Compare between different mer

# k = 3, run 10 times

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset, DataLoader
import random
import matplotlib.pyplot as plt
from itertools import product
from collections import defaultdict

# Constants
k = 3
input_fasta = 'data1/fungi_ITS_sorted90.fasta'
cleaned_fasta = 'data1/fungi_ITS_cleaned.fasta'
kmer_vector_file = 'data1/fungi_ITS_kmer_vector.txt'
filtered_vector_file = 'data1/fungi_ITS_kmer_vector_filtered.txt'
num_epochs = 100
num_runs = 10

# Helper functions
def generate_kmers(k):
    return [''.join(p) for p in product('ACGT', repeat=k)]

def kmer_vector(seq, k, kmer_index):
    vector = [0] * len(kmer_index)
    for i in range(len(seq) - k + 1):
        kmer = seq[i:i+k]
        if kmer in kmer_index:
            vector[kmer_index[kmer]] += 1
    return vector

class KmerDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return torch.tensor(self.data[idx]), self.labels[idx]

class CNNModel(nn.Module):
    def __init__(self, input_dim, num_classes):
        super().__init__()
        self.model = nn.Sequential(
            nn.Conv1d(1, 32, 7, padding='same'), nn.BatchNorm1d(32), nn.LeakyReLU(), nn.MaxPool1d(2),
            nn.Conv1d(32, 64, 7, padding='same'), nn.BatchNorm1d(64), nn.LeakyReLU(), nn.MaxPool1d(2),
            nn.Conv1d(64, 128, 7, padding='same'), nn.BatchNorm1d(128), nn.LeakyReLU(), nn.MaxPool1d(2),
            nn.Flatten(),
            nn.Linear((input_dim//8)*128, 1024), nn.LeakyReLU(), nn.Dropout(0.2),
            nn.Linear(1024, 256), nn.LeakyReLU(),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        return self.model(x.unsqueeze(1))

# Step 1: Clean FASTA headers
with open(input_fasta, 'r') as infile, open(cleaned_fasta, 'w') as outfile:
    for line in infile:
        if line.startswith('>'):
            parts = line.strip().split()
            outfile.write(f'>{parts[1]}\n' if len(parts) > 1 else line)
        else:
            outfile.write(line)

# Step 2: k-mer vectorization
kmers = generate_kmers(k)
kmer_index = {kmer: idx for idx, kmer in enumerate(kmers)}

with open(cleaned_fasta, 'r') as infile, open(kmer_vector_file, 'w') as outfile:
    current_sequence, header = '', ''
    for line in infile:
        if line.startswith('>'):
            if current_sequence:
                vector = kmer_vector(current_sequence, k, kmer_index)
                outfile.write(f"{header}\t{' '.join(map(str, vector))}\n")
            header = line.strip()
            current_sequence = ''
        else:
            current_sequence += line.strip()
    if current_sequence:
        vector = kmer_vector(current_sequence, k, kmer_index)
        outfile.write(f"{header}\t{' '.join(map(str, vector))}\n")

# Step 2: Filter sequences by genus frequency
genera_count = defaultdict(int)
with open(kmer_vector_file, 'r') as f:
    for line in f:
        genus = line.split()[0][1:]
        genera_count[genus] += 1

with open(kmer_vector_file, 'r') as infile, open(filtered_vector_file, 'w') as outfile:
    for line in infile:
        genus = line.split()[0][1:]
        if genera_count[genus] >= 10:
            outfile.write(line)

# Main Experiment Loop
best_accuracies = []

for run in range(num_runs):
    data_by_class = defaultdict(list)
    with open(filtered_vector_file, 'r') as f:
        for line in f:
            label = line.split('\t')[0][1:]
            data_by_class[label].append(line.strip())

    train_data, test_data = [], []
    for samples in data_by_class.values():
        test_sample = random.choice(samples)
        test_data.append(test_sample)
        train_data.extend(s for s in samples if s != test_sample)

    train_vectors = [list(map(int, line.split('\t')[1].split())) for line in train_data]
    train_labels = [line.split('\t')[0][1:] for line in train_data]

    test_vectors = [list(map(int, line.split('\t')[1].split())) for line in test_data]
    test_labels = [line.split('\t')[0][1:] for line in test_data]

    label_encoder = LabelEncoder().fit(train_labels)
    train_labels_encoded = label_encoder.transform(train_labels)
    test_labels_encoded = label_encoder.transform(test_labels)

    train_loader = DataLoader(KmerDataset(train_vectors, train_labels_encoded), batch_size=32, shuffle=True)
    test_loader = DataLoader(KmerDataset(test_vectors, test_labels_encoded), batch_size=32)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = CNNModel(len(train_vectors[0]), len(label_encoder.classes_)).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=0.001)

    max_accuracy = 0
    for epoch in range(num_epochs):
        model.train()
        for data, labels in train_loader:
            data, labels = data.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(data)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

        model.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for data, labels in test_loader:
                data, labels = data.to(device), labels.to(device)
                outputs = model(data)
                _, predicted = torch.max(outputs, 1)
                correct += (predicted == labels).sum().item()
                total += labels.size(0)

        accuracy = correct / total
        if accuracy > max_accuracy:
            max_accuracy = accuracy

    best_accuracies.append(max_accuracy)

avg_accuracy = sum(best_accuracies) / len(best_accuracies)
print(f'Average Highest Accuracy over {num_runs} runs: {avg_accuracy:.4f}')


# K = 5, 10 times running

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset, DataLoader
import random
import matplotlib.pyplot as plt
from itertools import product
from collections import defaultdict

# Constants
k = 5
input_fasta = 'data1/fungi_ITS_sorted90.fasta'
cleaned_fasta = 'data1/fungi_ITS_cleaned.fasta'
kmer_vector_file = 'data1/fungi_ITS_kmer_vector.txt'
filtered_vector_file = 'data1/fungi_ITS_kmer_vector_filtered.txt'
num_epochs = 100
num_runs = 10

# Helper functions
def generate_kmers(k):
    return [''.join(p) for p in product('ACGT', repeat=k)]

def kmer_vector(seq, k, kmer_index):
    vector = [0] * len(kmer_index)
    for i in range(len(seq) - k + 1):
        kmer = seq[i:i+k]
        if kmer in kmer_index:
            vector[kmer_index[kmer]] += 1
    return vector

class KmerDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return torch.tensor(self.data[idx]), self.labels[idx]

class CNNModel(nn.Module):
    def __init__(self, input_dim, num_classes):
        super().__init__()
        self.model = nn.Sequential(
            nn.Conv1d(1, 32, 7, padding='same'), nn.BatchNorm1d(32), nn.LeakyReLU(), nn.MaxPool1d(2),
            nn.Conv1d(32, 64, 7, padding='same'), nn.BatchNorm1d(64), nn.LeakyReLU(), nn.MaxPool1d(2),
            nn.Conv1d(64, 128, 7, padding='same'), nn.BatchNorm1d(128), nn.LeakyReLU(), nn.MaxPool1d(2),
            nn.Flatten(),
            nn.Linear((input_dim//8)*128, 1024), nn.LeakyReLU(), nn.Dropout(0.2),
            nn.Linear(1024, 256), nn.LeakyReLU(),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        return self.model(x.unsqueeze(1))

# Step 1: Clean FASTA headers
with open(input_fasta, 'r') as infile, open(cleaned_fasta, 'w') as outfile:
    for line in infile:
        if line.startswith('>'):
            parts = line.strip().split()
            outfile.write(f'>{parts[1]}\n' if len(parts) > 1 else line)
        else:
            outfile.write(line)

# Step 2: k-mer vectorization
kmers = generate_kmers(k)
kmer_index = {kmer: idx for idx, kmer in enumerate(kmers)}

with open(cleaned_fasta, 'r') as infile, open(kmer_vector_file, 'w') as outfile:
    current_sequence, header = '', ''
    for line in infile:
        if line.startswith('>'):
            if current_sequence:
                vector = kmer_vector(current_sequence, k, kmer_index)
                outfile.write(f"{header}\t{' '.join(map(str, vector))}\n")
            header = line.strip()
            current_sequence = ''
        else:
            current_sequence += line.strip()
    if current_sequence:
        vector = kmer_vector(current_sequence, k, kmer_index)
        outfile.write(f"{header}\t{' '.join(map(str, vector))}\n")

# Step 2: Filter sequences by genus frequency
genera_count = defaultdict(int)
with open(kmer_vector_file, 'r') as f:
    for line in f:
        genus = line.split()[0][1:]
        genera_count[genus] += 1

with open(kmer_vector_file, 'r') as infile, open(filtered_vector_file, 'w') as outfile:
    for line in infile:
        genus = line.split()[0][1:]
        if genera_count[genus] >= 10:
            outfile.write(line)

# Main Experiment Loop
best_accuracies = []

for run in range(num_runs):
    data_by_class = defaultdict(list)
    with open(filtered_vector_file, 'r') as f:
        for line in f:
            label = line.split('\t')[0][1:]
            data_by_class[label].append(line.strip())

    train_data, test_data = [], []
    for samples in data_by_class.values():
        test_sample = random.choice(samples)
        test_data.append(test_sample)
        train_data.extend(s for s in samples if s != test_sample)

    train_vectors = [list(map(int, line.split('\t')[1].split())) for line in train_data]
    train_labels = [line.split('\t')[0][1:] for line in train_data]

    test_vectors = [list(map(int, line.split('\t')[1].split())) for line in test_data]
    test_labels = [line.split('\t')[0][1:] for line in test_data]

    label_encoder = LabelEncoder().fit(train_labels)
    train_labels_encoded = label_encoder.transform(train_labels)
    test_labels_encoded = label_encoder.transform(test_labels)

    train_loader = DataLoader(KmerDataset(train_vectors, train_labels_encoded), batch_size=32, shuffle=True)
    test_loader = DataLoader(KmerDataset(test_vectors, test_labels_encoded), batch_size=32)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = CNNModel(len(train_vectors[0]), len(label_encoder.classes_)).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=0.001)

    max_accuracy = 0
    for epoch in range(num_epochs):
        model.train()
        for data, labels in train_loader:
            data, labels = data.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(data)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

        model.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for data, labels in test_loader:
                data, labels = data.to(device), labels.to(device)
                outputs = model(data)
                _, predicted = torch.max(outputs, 1)
                correct += (predicted == labels).sum().item()
                total += labels.size(0)

        accuracy = correct / total
        if accuracy > max_accuracy:
            max_accuracy = accuracy

    best_accuracies.append(max_accuracy)

avg_accuracy = sum(best_accuracies) / len(best_accuracies)
print(f'Average Highest Accuracy over {num_runs} runs: {avg_accuracy:.4f}')


# K = 6, 10 times running

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset, DataLoader
import random
import matplotlib.pyplot as plt
from itertools import product
from collections import defaultdict

# Constants
k = 6
input_fasta = 'data1/fungi_ITS_sorted90.fasta'
cleaned_fasta = 'data1/fungi_ITS_cleaned.fasta'
kmer_vector_file = 'data1/fungi_ITS_kmer_vector.txt'
filtered_vector_file = 'data1/fungi_ITS_kmer_vector_filtered.txt'
num_epochs = 100
num_runs = 10

# Helper functions
def generate_kmers(k):
    return [''.join(p) for p in product('ACGT', repeat=k)]

def kmer_vector(seq, k, kmer_index):
    vector = [0] * len(kmer_index)
    for i in range(len(seq) - k + 1):
        kmer = seq[i:i+k]
        if kmer in kmer_index:
            vector[kmer_index[kmer]] += 1
    return vector

class KmerDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return torch.tensor(self.data[idx]), self.labels[idx]

class CNNModel(nn.Module):
    def __init__(self, input_dim, num_classes):
        super().__init__()
        self.model = nn.Sequential(
            nn.Conv1d(1, 32, 7, padding='same'), nn.BatchNorm1d(32), nn.LeakyReLU(), nn.MaxPool1d(2),
            nn.Conv1d(32, 64, 7, padding='same'), nn.BatchNorm1d(64), nn.LeakyReLU(), nn.MaxPool1d(2),
            nn.Conv1d(64, 128, 7, padding='same'), nn.BatchNorm1d(128), nn.LeakyReLU(), nn.MaxPool1d(2),
            nn.Flatten(),
            nn.Linear((input_dim//8)*128, 1024), nn.LeakyReLU(), nn.Dropout(0.2),
            nn.Linear(1024, 256), nn.LeakyReLU(),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        return self.model(x.unsqueeze(1))

# Step 1: Clean FASTA headers
with open(input_fasta, 'r') as infile, open(cleaned_fasta, 'w') as outfile:
    for line in infile:
        if line.startswith('>'):
            parts = line.strip().split()
            outfile.write(f'>{parts[1]}\n' if len(parts) > 1 else line)
        else:
            outfile.write(line)

# Step 2: k-mer vectorization
kmers = generate_kmers(k)
kmer_index = {kmer: idx for idx, kmer in enumerate(kmers)}

with open(cleaned_fasta, 'r') as infile, open(kmer_vector_file, 'w') as outfile:
    current_sequence, header = '', ''
    for line in infile:
        if line.startswith('>'):
            if current_sequence:
                vector = kmer_vector(current_sequence, k, kmer_index)
                outfile.write(f"{header}\t{' '.join(map(str, vector))}\n")
            header = line.strip()
            current_sequence = ''
        else:
            current_sequence += line.strip()
    if current_sequence:
        vector = kmer_vector(current_sequence, k, kmer_index)
        outfile.write(f"{header}\t{' '.join(map(str, vector))}\n")

# Step 2: Filter sequences by genus frequency
genera_count = defaultdict(int)
with open(kmer_vector_file, 'r') as f:
    for line in f:
        genus = line.split()[0][1:]
        genera_count[genus] += 1

with open(kmer_vector_file, 'r') as infile, open(filtered_vector_file, 'w') as outfile:
    for line in infile:
        genus = line.split()[0][1:]
        if genera_count[genus] >= 10:
            outfile.write(line)

# Main Experiment Loop
best_accuracies = []

for run in range(num_runs):
    data_by_class = defaultdict(list)
    with open(filtered_vector_file, 'r') as f:
        for line in f:
            label = line.split('\t')[0][1:]
            data_by_class[label].append(line.strip())

    train_data, test_data = [], []
    for samples in data_by_class.values():
        test_sample = random.choice(samples)
        test_data.append(test_sample)
        train_data.extend(s for s in samples if s != test_sample)

    train_vectors = [list(map(int, line.split('\t')[1].split())) for line in train_data]
    train_labels = [line.split('\t')[0][1:] for line in train_data]

    test_vectors = [list(map(int, line.split('\t')[1].split())) for line in test_data]
    test_labels = [line.split('\t')[0][1:] for line in test_data]

    label_encoder = LabelEncoder().fit(train_labels)
    train_labels_encoded = label_encoder.transform(train_labels)
    test_labels_encoded = label_encoder.transform(test_labels)

    train_loader = DataLoader(KmerDataset(train_vectors, train_labels_encoded), batch_size=32, shuffle=True)
    test_loader = DataLoader(KmerDataset(test_vectors, test_labels_encoded), batch_size=32)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = CNNModel(len(train_vectors[0]), len(label_encoder.classes_)).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=0.001)

    max_accuracy = 0
    for epoch in range(num_epochs):
        model.train()
        for data, labels in train_loader:
            data, labels = data.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(data)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

        model.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for data, labels in test_loader:
                data, labels = data.to(device), labels.to(device)
                outputs = model(data)
                _, predicted = torch.max(outputs, 1)
                correct += (predicted == labels).sum().item()
                total += labels.size(0)

        accuracy = correct / total
        if accuracy > max_accuracy:
            max_accuracy = accuracy

    best_accuracies.append(max_accuracy)

avg_accuracy = sum(best_accuracies) / len(best_accuracies)
print(f'Average Highest Accuracy over {num_runs} runs: {avg_accuracy:.4f}')
